## Homework 5: Weather Prediction

### Jacqueline Beechert, jbeechert@berkeley.edu

Due Tuesday March 8, 2022 @ 5pm

## 1. Use the data supplied of airport codes (ICAO_airports.csv) and top airports (top_airports.csv) to make a table of 50 most travelled airports in the US and containing relevant information, like name, wikipedia link, latitude, longitude

In [269]:
import pandas as pd
import numpy as np
import sqlite3

In [270]:
icao_df = pd.read_csv("hw_5_data/ICAO_airports.csv")
top_df = pd.read_csv("hw_5_data/top_airports.csv")

In [282]:
icao_df.head(5)

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total Rf Heliport,40.070801,-74.933601,11.0,NaN,US,US-PA,Bensalem,no,00A,NaN,00A,NaN,NaN,NaN
1,6524,00AK,small_airport,Lowell Field,59.949200,-151.695999,450.0,NaN,US,US-AK,Anchor Point,no,00AK,NaN,00AK,NaN,NaN,NaN
2,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,00AL,NaN,00AL,NaN,NaN,NaN
3,6526,00AR,heliport,Newport Hospital & Clinic Heliport,35.608700,-91.254898,237.0,NaN,US,US-AR,Newport,no,00AR,NaN,00AR,NaN,NaN,NaN
4,6527,00AZ,small_airport,Cordes Airport,34.305599,-112.165001,3810.0,NaN,US,US-AZ,Cordes,no,00AZ,NaN,00AZ,NaN,NaN,NaN


In [283]:
top_50_df = top_df.nlargest(50, "Enplanements")
top_50_airport_names = top_50_df["Airport"].values
print(top_50_airport_names[:5])

['Hartsfield-Jackson Atlanta International Airport'
 "Chicago O'Hare International Airport"
 'Los Angeles International Airport'
 'Dallas/Fort Worth International Airport' 'Denver International Airport']


In [362]:
# make icao table

!rm hw5.db
connection = sqlite3.connect("hw5.db")

cursor = connection.cursor()

sql_cmd = """CREATE TABLE IF NOT EXISTS icao (id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT, ident TEXT, wikipedia_link TEXT, latitude FLOAT, longitude FLOAT)"""

cursor.execute(sql_cmd)

for i, row in icao_df.iterrows():

    name = row['name']
    ident = row['ident']
    wiki = str(row["wikipedia_link"])
    latitude = row["latitude_deg"]
    longitude = row["longitude_deg"] 

    data = (name, ident, wiki, latitude, longitude)

    # "or ignore" prevents inserting values if the primary key already exists
    sql_cmd = ("INSERT OR IGNORE INTO icao (name, ident, wikipedia_link, latitude, longitude) VALUES" + str(data))
    cursor.execute(sql_cmd)
    
connection.commit()  

In [363]:
# make top 50 table

cursor = connection.cursor()

sql_cmd = """CREATE TABLE IF NOT EXISTS top50 (id INTEGER PRIMARY KEY AUTOINCREMENT,
    city TEXT, airport TEXT, icao TEXT, enplanements INT)"""

cursor.execute(sql_cmd)

for i, row in top_50_df.iterrows():
    
    city = str(row['City'])
    airport = str(row["Airport"])
    icao = str(row["ICAO"])
    enplanements = int(row["Enplanements"])
    
    data = (city, airport, icao, enplanements)
    
    sql_cmd = ("INSERT OR IGNORE INTO top50 (city, airport, icao, enplanements) VALUES " + str(data))
    cursor.execute(sql_cmd)
    
    
connection.commit()

In [364]:
sql_cmd = f"""SELECT *, top50.icao FROM icao, top50 WHERE icao.ident == top50.icao"""

In [365]:
cursor.execute(sql_cmd)

In [366]:
db_info = cursor.fetchall()

cnt = 0
for entry in db_info: 
    cnt += 1
print(cnt)

50


In [367]:
cursor = connection.cursor()

sql_cmd = """CREATE TABLE IF NOT EXISTS icao_top50 (id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT, ident TEXT, wikipedia_link TEXT, latitude FLOAT, longitude FLOAT)"""

cursor.execute(sql_cmd)

for entry in db_info:

    # "or ignore" prevents inserting values if the primary key already exists
    sql_cmd = ("INSERT OR IGNORE INTO icao_top50 (name, ident, wikipedia_link, latitude, longitude) VALUES" 
               + str(entry[1:6]))
    cursor.execute(sql_cmd)
    
connection.commit()  

## 2. Build another table that will hold historical weather information: min/max temperature, relative humidity, and precipitation

In [ ]:
# make weather table
connection = sqlite3.connect("weather.db")

cursor = connection.cursor()

sql_cmd = """CREATE TABLE weather (id INTEGER PRIMARY KEY AUTOINCREMENT,
              lat FLOAT, lon FLOAT, max_temp FLOAT, min_temp FLOAT, 
              max_hum FLOAT, min_hum FLOAT, precipitation FLOAT)"""

cursor.execute(sql_cmd)

## 3. Using the data from the xarray/netCDF4 example in class, populate the database from 1990-2000

In [372]:
import xarray as xr

In [396]:
def select_data(path):
    
    # Open the data set as xarray
    xr_ = xr.open_dataset(path)
    
    # Select data between 1990-2000
    xr_ = xr_.sel(time=slice('1990', '2000')).load()
    
    return xr_

In [398]:
# Data path from lecture
max_temp_path = "http://thredds.northwestknowledge.net:8080/thredds/dodsC/agg_macav2metdata_tasmax_BNU-ESM_r1i1p1_historical_1950_2005_CONUS_monthly.nc" 

max_temp_xr = select_data(max_temp_path)

In [382]:
min_temp_path = "http://thredds.northwestknowledge.net:8080/thredds/dodsC/agg_macav2metdata_tasmin_BNU-ESM_r1i1p1_historical_1950_2005_CONUS_monthly.nc" 

min_temp_xr = select_data(min_temp_path)

In [377]:
max_hum_path = "http://thredds.northwestknowledge.net:8080/thredds/dodsC/agg_macav2metdata_rhsmax_BNU-ESM_r1i1p1_historical_1950_2005_CONUS_monthly.nc" 

max_hum_xr = select_data(max_hum_path)

In [376]:
min_hum_path = "http://thredds.northwestknowledge.net:8080/thredds/dodsC/agg_macav2metdata_rhsmin_BNU-ESM_r1i1p1_historical_1950_2005_CONUS_monthly.nc" 

min_hum_xr = select_data(min_hum_path)

In [373]:
precip_path = "http://thredds.northwestknowledge.net:8080/thredds/dodsC/agg_macav2metdata_pr_BNU-ESM_r1i1p1_historical_1950_2005_CONUS_monthly.nc" 

precip_xr = select_data(precip_path)

In [392]:
# Same latitudes and longitudes for all weather metrics
lats = max_temp_xr["air_temperature"]["lat"].values
lons = max_temp_xr["air_temperature"]["lon"].values
print(len(lats))

latitudes = np.array([round(x,2) for x in lats])
longitudes = np.array([round(x,2) for x in lons])

lat_idx = list(np.where(latitudes == 40.1)[0])
lon_idx = list(np.where(longitudes == -74.9+360)[0])

585


In [395]:
one_point = max_temp_xr["air_temperature"].sel(lat=lats[lat_idx],
                                               lon=lons[lon_idx])
print((one_point.values).shape)
print(len(one_point.values))

(132, 1, 1)
132


In [ ]:
n = len(precip_xr["precipitation"])

for i in range(n):
    lat = lats[i]
    lon = lons[i]